## Imports

In [1]:
from math import *
import numpy as np
import pandas as pd

#to read theb data
from pathlib import Path
import glob 

#for plots
import matplotlib.pyplot as plt
import seaborn as sns

#stats
from scipy.stats import kurtosis


------------------

## Reading the Data

In [2]:
# Reading the data
path = r'C:/Users/varsh/OneDrive/Desktop/Honours/Data'
all_files = glob.glob(path + "/*.csv")
path
all_files

#Concatanating the individual datasets into one dataframe
df = pd.concat((pd.read_csv(f) for f in all_files))

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4302 entries, 0 to 339
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Product code                          4302 non-null   object 
 1   Bureau of Meteorology station number  4302 non-null   int64  
 2   Year                                  4302 non-null   int64  
 3   Month                                 4302 non-null   int64  
 4   Mean maximum temperature (°C)         4302 non-null   float64
 5   Quality                               4302 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 235.3+ KB


------------------

## Data Cleaning

In [3]:
#renaming columns 
df = df.rename(columns = {'Bureau of Meteorology station number':'stationNumber',
                          'Mean maximum temperature (°C)': 'meanMaxTemp',
                         'Year':'year',
                         'Month':'month'})

In [4]:
#recoding month numbers to their names
df["month"] = df["month"].replace(np.arange(1,13), ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sept", 
                                                   "Oct", "Nov", "Dec"])

In [5]:
#setting the month columns as a categorical and imposing an order so it is automatically arranged sequentially,
#this is particularly useful for charts
#this will also allow us to see any missing data
df["month"] = pd.Categorical(df["month"], 
                      categories=["Jan", "Feb", "Mar","Apr", "May", "Jun", "Jul", "Aug", "Sept", 
                      "Oct", "Nov","Dec"],
                      ordered=True)

In [6]:
#use the station number to add a locations column 
df['stationNumber'].unique()

array([66062, 66124, 66137, 66214, 67033, 67105], dtype=int64)

In [7]:
#adding a location column that generalises the station location as Sydney, Richmond or Bankstown
df['location'] = ['Sydney' if (x == 66062 or x == 66214)  else 
                  'Richmond' if (x == 67105 or x == 67033) else 
                  'Parramatta' if (x == 66124) else
                  "Bankstown" for x in df['stationNumber']]


In [8]:
df.groupby('location')["year"].count()

location
Bankstown      648
Parramatta     654
Richmond      1008
Sydney        1992
Name: year, dtype: int64

### Dealing with Duplicates

In [9]:
#as the data for Sydney and Richmond both come from 2 stations, there is an overlap in data for some months.
#for example, in 2017:
dupes = df[df.duplicated(subset=['location','year','month'], keep=False)]
dupes[dupes.year == 2017]

,Product code,stationNumber,year,month,meanMaxTemp,Quality,location
1905,IDCJAC0002,66062,2017,Oct,24.0,Y,Sydney
1906,IDCJAC0002,66062,2017,Nov,23.8,Y,Sydney
1907,IDCJAC0002,66062,2017,Dec,27.8,Y,Sydney
0,IDCJAC0002,66214,2017,Oct,24.9,Y,Sydney
1,IDCJAC0002,66214,2017,Nov,24.8,Y,Sydney
2,IDCJAC0002,66214,2017,Dec,28.6,Y,Sydney


In [10]:
#to overcome this, we will replace the duplicate rows with averages of the meanMaxTemp
df = df.groupby(['location','year','month']).mean().reset_index()

#no more duplicates!!
df[(df.year == 2017) & (df.location == "Sydney")]

,location,year,month,stationNumber,meanMaxTemp
7800,Sydney,2017,Jan,66062.0,29.60
7801,Sydney,2017,Feb,66062.0,28.50
7802,Sydney,2017,Mar,66062.0,25.60
7803,Sydney,2017,Apr,66062.0,23.80
7804,Sydney,2017,May,66062.0,21.20
7805,Sydney,2017,Jun,66062.0,18.20
7806,Sydney,2017,Jul,66062.0,19.10
7807,Sydney,2017,Aug,66062.0,19.50
7808,Sydney,2017,Sept,66062.0,23.30
7809,Sydney,2017,Oct,66138.0,24.45


In [11]:
#after combining the columns, some of the rows are out of order so we sort all the columns
df.sort_values(by=['location','year', 'month'], inplace=True, ignore_index=True)

### Defining the 2 Periods

For comparision, we will look at all of the datasets from 1967. They will be split into two periods:
* 1962 - 1991
* 1992 - 2021

This will also allow us to see the accelerated changes in the recent in the last climate period.

The Bankstown dataset begins at 1968, so we will have to adjust the periods for:
* 1969 - 1994
* 1995 - 2021

The Parramatta dataset begins at 1967, so we will have to adjust the periods for:
* 1968 - 1994
* 1995 - 2021

Summer years are defined by the year of the January month. For instance, summer 2021 captures the months Dec 2020 to Mar 2021.

In [12]:
#remove data before  1961 dec
df = df[df['year'] >= 1961]

#remove Bankstown data before  1968 dec
df = df[(df['location']!= "Bankstown") | (df['year'] >= 1968)]

#remove parramatta data before  1967 dec
df = df[(df['location']!= "Parramatta") | (df['year'] >= 1967)]


#define the summer years in new column sYear
months = df.month.unique()[:11]
for i in months:
    df["sYear"] = df.loc[df["month"] == i, "year"] 

df["sYear"].bfill(inplace = True)

In [13]:
#define a new column, period, which categorises the years into the 2 periods
df['period'] = df['sYear'].apply(lambda x: '1962-1991' if x <= 1991 else '1992-2021')

df.loc[df.location == "Bankstown",'period'] = df['sYear'].apply(lambda x: '1969-1994' if x <= 1994 else '1995-2021')
df.loc[df.location == "Parramatta",'period'] = df['sYear'].apply(lambda x: '1968-1994' if x <= 1994 else '1995-2021')

### Dealing with Missing Values

In [14]:
#find the missing data after Dec 1967
df[(df.meanMaxTemp.isna()) & ((df.year < 2022))]

,location,year,month,stationNumber,meanMaxTemp,sYear,period
1308,Bankstown,1968,Jan,NaN,NaN,1968.0,1969-1994
1309,Bankstown,1968,Feb,NaN,NaN,1968.0,1969-1994
1310,Bankstown,1968,Mar,NaN,NaN,1968.0,1969-1994
1311,Bankstown,1968,Apr,NaN,NaN,1968.0,1969-1994
1312,Bankstown,1968,May,NaN,NaN,1968.0,1969-1994
1313,Bankstown,1968,Jun,NaN,NaN,1968.0,1969-1994
1602,Bankstown,1992,Jul,NaN,NaN,1992.0,1969-1994
3264,Parramatta,1967,Jan,NaN,NaN,1967.0,1968-1994
3265,Parramatta,1967,Feb,NaN,NaN,1967.0,1968-1994
3266,Parramatta,1967,Mar,NaN,NaN,1967.0,1968-1994


In [15]:
#get the temperatures of the years before and after the NaN value and use the average of it to fill
df['meanMaxTemp'] = (((df.groupby(['location','month'], sort=False)['meanMaxTemp']
                       .apply(lambda x: x.bfill().add(x.ffill()).div(2)))))
    
#the missing months in 1967 don't get filled as there is no previous year data for it to backwards fill on. 
#For these months, we will take the averages from 1968/69 for the missing months.

#add a column call rollmean2 that calculates the rolling mean for 2 months of a year backwards. ie, Jan 1967 would have
# a rollmean value that is the average of Jan 1968 and 1969
df['rollmean2'] = (((df.groupby(['location','month'], sort=False)['meanMaxTemp']
                       .apply(lambda x: x.rolling(2,center=False,min_periods=1).mean().shift(-2)))))
#replace the remaining missing values with the rollmean ones
df['meanMaxTemp'] = df['meanMaxTemp'].fillna(df['rollmean2'])

#drop the rollmean2 column after
df.drop(columns = ['rollmean2'], inplace = True)

#remove any rows without data
df.dropna(axis = 0, subset = ['meanMaxTemp'], inplace = True)

In [16]:
df["periodNum"] = 1
df.loc[(((df.location == "Bankstown") |(df.location == "Parramatta")) & (df.period == "1995-2021")), "periodNum"] = 2

df.loc[((df.location != "Bankstown") & (df.period == "1992-2021")), "periodNum"] = 2

In [17]:
df = df[df.sYear < 2022]

In [18]:
#filter the dataframe to include only months from Dec - Mar
summerMonths = ['Dec','Jan','Feb','Mar']
summerData = df[(df['month'].isin(summerMonths)) & (df.sYear != 2022)].copy()

#this also picks up Jan, Feb and Mar from 1960 so we have to drop those 
i = summerData[((summerData.year == 1961) & (summerData.month != "Dec"))].index

#drop Jan, Feb and Mar from 1968 Bankstown data
j = summerData[((summerData.year == 1968) & (summerData.month != "Dec") & (summerData.location == "Bankstown"))].index

#drop Jan, Feb and Mar from 1967 Parramatta data
k = summerData[((summerData.year == 1967) & (summerData.month != "Dec") & (summerData.location == "Parramatta"))].index


summerData.drop(i, inplace = True)
summerData.drop(j, inplace = True)
summerData.drop(k, inplace = True)

In [19]:
#change the month to categorical so we can order the data from Dec - Mar, this is just for visualisation purposes
summerData["month"] = pd.Categorical(summerData["month"], 
                      categories=["Dec", "Jan", "Feb", "Mar"],
                      ordered=True)

summerData["location"] = pd.Categorical(summerData["location"], 
                      categories=["Sydney", "Bankstown","Parramatta", "Richmond"],
                      ordered=True)

In [20]:
summerDataSyd = summerData[summerData.location == "Sydney"]
summerDataBank = summerData[summerData.location == "Bankstown"]
summerDataRich = summerData[summerData.location == "Richmond"]
summerDataParra = summerData[summerData.location == "Parramatta"]

In [21]:
#export the datasets
df.to_csv('df.csv')

summerData.to_csv('summerData.csv')
summerDataSyd.to_csv('summerDataSyd.csv')
summerDataBank.to_csv('summerDataBank.csv')
summerDataRich.to_csv('summerDataRich.csv')
summerDataParra.to_csv('summerDataParra.csv')
